In [ ]:
# 세션 유지 코드
from IPython.display import clear_output
import threading, time

def keep_alive():
    for i in range(100000):
        time.sleep(60)
        clear_output(wait=True)
        print(f"Ping {i} ⏱️")

threading.Thread(target=keep_alive).start()

## Training

In [ ]:
# ==========================================
# 0. Install & Import Dependencies
# ==========================================
!pip install transformers datasets peft accelerate bitsandbytes --quiet

import torch
from datasets import load_from_disk
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    DataCollatorWithPadding,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

In [ ]:
# ==========================================
# 1. Load Dataset (Math binary classification)
# ==========================================
print("🔄 Loading dataset...")

from google.colab import drive
drive.mount('/content/drive')

dataset = load_from_disk("/content/drive/MyDrive/Gemma_FineTuning/math_eval_binary_dataset")

# Optional: 확인
print(dataset[0])

In [ ]:
# ==========================================
# 2. Load Model & Tokenizer
# ==========================================

from huggingface_hub import login
login(token="YOUR_HF_TOKEN")

model_id = "google/gemma-3n-E2B-it"  # Use the same model used during generation

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    quantization_config=quant_config,
    num_labels=2,  # Binary classification (correct/incorrect)
    device_map="auto"
)

In [ ]:
# ==========================================
# 3. LoRA Config
# ==========================================
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)

model = get_peft_model(base_model, lora_config)

In [ ]:
# ==========================================
# 4. Dataset Preprocessing (포맷 설정)
# ==========================================
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [ ]:
# ==========================================
# 5. TrainingArguments & Trainer
# ==========================================
training_args = TrainingArguments(
    output_dir="./gemma3n-math-binary",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_steps=25,
    save_steps=250,
    learning_rate=2e-4,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="linear",
    optim="paged_adamw_8bit",
    report_to=None,
    bf16=True,
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    peft_config=lora_config,
    data_collator=DataCollatorWithPadding(tokenizer)
)

In [ ]:
# ==========================================
# 6. Start Training
# ==========================================
print("🚀 Starting training...")
trainer.train()

In [ ]:
# ==========================================
# 7. Save to Drive & Hugging Face
# ==========================================
save_path = "/content/drive/MyDrive/math_eval_binary"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

# Push to Hugging Face
repo_id = "LeannaJ/math_evaluation"
model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)
print(f"✅ Model pushed to Hugging Face: https://huggingface.co/{repo_id}")
